# LCoE calculator for geospatial grid-based electrification modelling

#### A more detailed explanation of parameters and functions is availavle ([here]((https://github.com/alekordESA/Cluster_Grid_Extension/wiki)).

In [30]:
## This code has been developed in order to calculate the lcoe for grid extension processes under a geo-spatial content.
## Author: Alexandros Korkovelos
## Latest update: 30-10-2018

import pandas as pd
import numpy as np
from math import ceil, sqrt, pi, exp, log, copysign
import os
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm
import json
from IPython.display import display, Markdown, HTML

## General input parameters related to the transmission network

In [31]:
# Input parameters

HV_line_type = 69                  # kV
HV_line_cost = 110000              # $/km for 69kV
    
MV_line_type = 11                  # kV
MV_line_amperage_limit = 8.0       # Ampere (A)
MV_line_span = 25                  # km buffer area that can be reached by an MV line
MV_line_cost = 5000                # $/km  for 11-33 kV
MV_line_max_stretch = 50           # km length of MV line from main HV grid
#MV_increase_rate = 0.1            # (%)
    
LV_line_type = 0.240               # kV
LV_line_cost = 3000                # $/km for 0.4 kV
LV_line_max_length = 0.5           # km
    
service_Transf_type = 50           # kVa
service_Transf_cost = 3500         # $/unit of 50 kVa
max_nodes_per_serv_trans = 300     # maximum number of nodes served by each service transformer
MV_LV_sub_station_type = 400       # kVa
MV_LV_sub_station_cost = 10000     # $/unit of 50 kVa
HV_LV_sub_station_type = 1000      # kVa
HV_LV_sub_station_cost = 25000     # $/unit of 50 kVa
    
grid_fuel_cost = 0.076             # $/kWh
base_to_peak = 0.85                # for the sizing of needed capacity
trans_losses = 0.05                # (%)
distr_losses = 0.15                # (%)
trans_O_M = 0.03                   # (%) Operation and maintenance costs for T&D
disc_rate = 0.08                   # (%) discount rate
system_life = 30                   # years
con_cost_per_node = 100            # $

#simultaneous_usage = 0.06          # From (2)
power_factor = 0.9                 # From (1)
load_moment = 9643                 # for 50mm aluminum conductor under 5% voltage drop (kW m) (2)
project_life = 15                  # start - end year of analysis
fuel_cost = 0.076                  # $/kWh grid generating cost of electricity by the central grid
hours_per_year = 8760    

## Definition of a function that calculates the lcoe for certain inputs

In [41]:
def calc_lcoe(people, access_target, ppl_per_hh, productive_demand, productive_nodes, cluster_area, elec_dist):
    
# To prevent any div/0 error
    if people == 0:
        people = 0.00001 

    # Estimate total demand in settlement
    res_demand = people * access_target
    total_demand = res_demand + productive_demand
    total_nodes = (people/ppl_per_hh) + productive_nodes
    MV_tmp_length = 0
    No_of_LV_lines = 0
    No_of_MV_lines = 0
    No_of_HV_lines = 0
    No_of_HV_LV_subs = 0
    No_of_MV_LV_subs = 0
    
        # Estimate peak load in settlement
    average_load = total_demand/hours_per_year
    peak_load = average_load / base_to_peak
    print ("The peak load in this cluster is:", peak_load)
    print ("The area of this cluster is:", cluster_area)
        
        # Sizing HV/MV
    HV_to_MV_lines  = HV_line_cost/MV_line_cost
    max_MV_load = MV_line_amperage_limit*MV_line_type*HV_to_MV_lines
    MV_temp_length = 0
    print ("The max MV load is:", max_MV_load)
    
    if peak_load <= max_MV_load and MV_temp_length <= MV_line_max_stretch:
        MV_amperage = MV_LV_sub_station_type/MV_line_type
        No_of_MV_lines = ceil(peak_load/(MV_amperage*MV_line_type*(1-(trans_losses + distr_losses))))
        MV_km = elec_dist * No_of_MV_lines
        HV_km = 0
        MV_temp_length =+ elec_dist
        No_of_MV_LV_subs = 1  
    elif peak_load >= max_MV_load or MV_temp_length >= MV_line_max_stretch:
        HV_amperage = HV_LV_sub_station_type/HV_line_type
        No_of_HV_lines = ceil(peak_load/(HV_amperage*HV_line_type*(1-(trans_losses + distr_losses))))
        HV_km = elec_dist * No_of_HV_lines
        MV_km = 0
        MV_temp_length = 0
        No_of_HV_LV_subs = 1
    # Unecessary here but keep just in case
    else:                           
        HV_km = 0
        No_of_MV_LV_subs = 0
        MV_km = 0
        No_of_HV_LV_subs = 0
    
        # Sizing service transformers in settlement 
    Smax = peak_load/power_factor
    max_tranformer_area = pi * LV_line_max_length ** 2
    
    no_of_service_transf = ceil(max(Smax/service_Transf_type, total_nodes/max_nodes_per_serv_trans, cluster_area / max_tranformer_area))
    transformer_radius = ((cluster_area / no_of_service_transf) / pi) ** 0.5
    transformer_nodes = total_nodes / no_of_service_transf
    transformer_load = peak_load / no_of_service_transf
    cluster_radius = (cluster_area / pi) ** 0.5
    
    # Sizing LV lines in settlement
    if 2/3 * cluster_radius * transformer_load * 1000 < load_moment:
        cluster_lv_lines_length = 2/3 * cluster_radius *  no_of_service_transf
        cluster_mv_lines_length = 0
    else:
        cluster_lv_lines_length = 0
        cluster_mv_lines_length = 2/3 * cluster_radius *  no_of_service_transf
    
    transformer_lv_lines_length = 2/3 * transformer_radius * total_nodes
    
    LV_km = cluster_lv_lines_length + transformer_lv_lines_length
    MV_km += cluster_mv_lines_length  

    # LV_area_limit = cluster_area/(no_of_service_transf*LV_line_max_length*LV_line_unit)
    # LV_line_amperage = service_Transf_type/LV_line_type
    # LV_capacity_limit = (peak_load/(LV_line_type*LV_line_amperage*(1-distr_losses)))
    # No_of_LV_lines = max(LV_area_limit,LV_capacity_limit)
    # LV_km = ceil(No_of_LV_lines)*LV_line_unit
    
    print ("Cluster_distance:", MV_temp_length, "\nHV_km:", HV_km, "\nMV_km:", MV_km, "\nLV_km:", LV_km, "\nServ_Trans:", no_of_service_transf, "\nnodes:", total_nodes)
    
    # print ("LV_line_unit:", LV_line_unit, "\nLV_area_limit:",  LV_area_limit, "\nLV_line_amperage", LV_line_amperage, "\nLV_capacity_limit:", LV_capacity_limit, "\nNo_of_LV_lines:", No_of_LV_lines)

    print ("HV_cost:", HV_km * HV_line_cost + (No_of_HV_LV_subs*HV_LV_sub_station_cost),
      "\nMV_cost:", MV_km * MV_line_cost + (No_of_MV_LV_subs*MV_LV_sub_station_cost),
      "\nLV_cost:", LV_km * LV_line_cost,
      "\nServ_Trans_cost:", no_of_service_transf * service_Transf_cost,
      "\nnode_Connection_cost:", total_nodes * con_cost_per_node)
   
    ## Calculation of the investment cost
    td_investment_cost = (HV_km * HV_line_cost + (No_of_HV_LV_subs*HV_LV_sub_station_cost)) + \
                         (MV_km * MV_line_cost + (No_of_MV_LV_subs*MV_LV_sub_station_cost)) + \
                         (LV_km * LV_line_cost) + \
                         (no_of_service_transf * service_Transf_cost) +\
                         (total_nodes * con_cost_per_node)
    td_om_cost = td_investment_cost * trans_O_M
    total_investment_cost = td_investment_cost
    total_om_cost = td_om_cost
    print ("The total investment cost is estimated at:", total_investment_cost, "$")
    
    ## lcoe calculation 
    
    if system_life<project_life:
        reinvest_year = system_life
    else:
        reinvest_year = 0
        
    year = np.arange(project_life)
    generation_per_year = average_load*hours_per_year
    el_gen = generation_per_year * np.ones(project_life)
    el_gen[0] = 0
    discount_factor = (1 + disc_rate) ** year
    investments = np.zeros(project_life)
    investments[0] = total_investment_cost
    if reinvest_year:
        investments[reinvest_year] = total_investment_cost
    
    salvage = np.zeros(project_life)
    used_life = project_life
    if reinvest_year is not 0:
        used_life = project_life - system_life
        salvage[-1] = total_investment_cost * (1 - used_life / system_life)
    
    operation_and_maintenance = total_om_cost * np.ones(project_life)
    operation_and_maintenance[0] = 0
    fuel = el_gen * fuel_cost
    fuel[0] = 0
    
    discounted_costs = (investments + operation_and_maintenance + fuel - salvage) / discount_factor
    discounted_generation = el_gen / discount_factor
    lcoe = np.sum(discounted_costs) / np.sum(discounted_generation)
    
    print ("The lcoe is:", lcoe, "\n")
    return lcoe

## Dynamic parameters that depend on the cluster

In [42]:
# Read the cluster characteristics from a csv file
df = pd.read_csv("Malawi_Clusters.csv")

In [43]:
# Dynamic inputs

people = df["Pop"]                                         # people
access_target = df["ResidentialDemandTierCustom"]          # kWh/pp/year
ppl_per_hh = df["NumPeoplePerHH"]                          # people per household
productive_demand = df["Productive_demand"]                # kWh/year/settlement
productive_nodes = df["Productive_nodes"]                  # other demand nodes
cluster_area = df["GridCellArea"]                          # km2
elec_dist = df["PlannedHVLineDist"]                        # distance of cluster area from closest electrified settlement

## Run the function with the above inputs

In [44]:
#logging.info('Calculate grid LCOE')

df["lcoe"] = df.apply(lambda row: calc_lcoe(people=row["Pop"],
                                            access_target=row["ResidentialDemandTierCustom"],
                                            ppl_per_hh=row["NumPeoplePerHH"],
                                            productive_demand=row["Productive_demand"],
                                            productive_nodes=row["Productive_nodes"],
                                            cluster_area=row["GridCellArea"],
                                            elec_dist = row["PlannedHVLineDist"]),axis=1)

The peak load in this cluster is: 1977.4573629749295
The area of this cluster is: 584.5470799999999
The max MV load is: 1936.0
Cluster_distance: 0 
HV_km: 0.56391 
MV_km: 6774.8521993721215 
LV_km: 13315.213089564319 
Serv_Trans: 745 
nodes: 39965.26574545455
HV_cost: 87030.1 
MV_cost: 33874260.99686061 
LV_cost: 39945639.268692955 
Serv_Trans_cost: 2607500 
node_Connection_cost: 3996526.5745454547
The total investment cost is estimated at: 80510956.94009902 $
The lcoe is: 0.9032841862492551 

The peak load in this cluster is: 89769.0837880987
The area of this cluster is: 559.74321
The max MV load is: 1936.0
Cluster_distance: 0 
HV_km: 5.56412 
MV_km: 17752.97418856395 
LV_km: 48113.46789302265 
Serv_Trans: 1995 
nodes: 241495.9708
HV_cost: 637053.2 
MV_cost: 88764870.94281974 
LV_cost: 144340403.67906794 
Serv_Trans_cost: 6982500 
node_Connection_cost: 24149597.080000002
The total investment cost is estimated at: 264874424.90188769 $
The lcoe is: 0.1359542668328478 

The peak load in 

LV_km: 446.90443609946595 
Serv_Trans: 6 
nodes: 1419.4242472727271
HV_cost: 0 
MV_cost: 46250.538676710334 
LV_cost: 1340713.3082983978 
Serv_Trans_cost: 21000 
node_Connection_cost: 141942.4247272727
The total investment cost is estimated at: 1549906.2717023809 $
The lcoe is: 0.2413680418865929 

The peak load in this cluster is: 32.39588137074179
The area of this cluster is: 4.16685
The max MV load is: 1936.0
Cluster_distance: 17.91864 
HV_km: 0 
MV_km: 17.91864 
LV_km: 110.67118329510824 
Serv_Trans: 6 
nodes: 338.3825109090909
HV_cost: 0 
MV_cost: 99593.2 
LV_cost: 332013.5498853247 
Serv_Trans_cost: 21000 
node_Connection_cost: 33838.25109090909
The total investment cost is estimated at: 486445.00097623386 $
The lcoe is: 0.3811060412921427 

The peak load in this cluster is: 33.665874219878766
The area of this cluster is: 4.14652
The max MV load is: 1936.0
Cluster_distance: 24.56232 
HV_km: 0 
MV_km: 24.56232 
LV_km: 101.1862054315783 
Serv_Trans: 6 
nodes: 308.91255090909095
HV_

node_Connection_cost: 42926.089636363635
The total investment cost is estimated at: 522411.08531826467 $
The lcoe is: 0.2200819740163407 

The peak load in this cluster is: 25.948775991418525
The area of this cluster is: 3.83206
The max MV load is: 1936.0
Cluster_distance: 11.915230000000001 
HV_km: 0 
MV_km: 11.915230000000001 
LV_km: 115.42543591569074 
Serv_Trans: 5 
nodes: 339.35884363636364
HV_cost: 0 
MV_cost: 69576.15000000001 
LV_cost: 346276.30774707225 
Serv_Trans_cost: 17500 
node_Connection_cost: 33935.88436363637
The total investment cost is estimated at: 467288.34211070865 $
The lcoe is: 0.4419105504216075 

The peak load in this cluster is: 273.0998444782177
The area of this cluster is: 3.8272800000000005
The max MV load is: 1936.0
Cluster_distance: 0.96619 
HV_km: 0 
MV_km: 6.11701698855162 
LV_km: 379.0591705750752 
Serv_Trans: 7 
nodes: 1362.9411545454545
HV_cost: 0 
MV_cost: 40585.0849427581 
LV_cost: 1137177.5117252255 
Serv_Trans_cost: 24500 
node_Connection_cost: 

The peak load in this cluster is: 63.94958665006902
The area of this cluster is: 2.5928400000000003
The max MV load is: 1936.0
Cluster_distance: 12.62989 
HV_km: 0 
MV_km: 15.05248967662697 
LV_km: 139.68132110585486 
Serv_Trans: 4 
nodes: 461.26092545454543
HV_cost: 0 
MV_cost: 85262.44838313485 
LV_cost: 419043.9633175646 
Serv_Trans_cost: 14000 
node_Connection_cost: 46126.09254545454
The total investment cost is estimated at: 564432.504246154 $
The lcoe is: 0.2553416398003538 

The peak load in this cluster is: 108.4740021220808
The area of this cluster is: 2.58299
The max MV load is: 1936.0
Cluster_distance: 4.1148 
HV_km: 0 
MV_km: 6.532793662947482 
LV_km: 186.0555712656854 
Serv_Trans: 4 
nodes: 615.5700872727273
HV_cost: 0 
MV_cost: 42663.96831473741 
LV_cost: 558166.7137970561 
Serv_Trans_cost: 14000 
node_Connection_cost: 61557.008727272725
The total investment cost is estimated at: 676387.6908390663 $
The lcoe is: 0.20270008149914964 

The peak load in this cluster is: 69.3

The total investment cost is estimated at: 221635.23878861236 $
The lcoe is: 0.41063262258662503 

The peak load in this cluster is: 45.89260094998117
The area of this cluster is: 2.3511900000000003
The max MV load is: 1936.0
Cluster_distance: 15.045229999999998 
HV_km: 0 
MV_km: 15.045229999999998 
LV_km: 138.36030631125553 
Serv_Trans: 3 
nodes: 410.32628
HV_cost: 0 
MV_cost: 85226.15 
LV_cost: 415080.9189337666 
Serv_Trans_cost: 10500 
node_Connection_cost: 41032.628
The total investment cost is estimated at: 551839.6969337666 $
The lcoe is: 0.32033018788098866 

The peak load in this cluster is: 73.39979581766718
The area of this cluster is: 2.34152
The max MV load is: 1936.0
Cluster_distance: 1.26203 
HV_km: 0 
MV_km: 2.9886787363642546 
LV_km: 135.9159218549605 
Serv_Trans: 3 
nodes: 409.02346363636366
HV_cost: 0 
MV_cost: 24943.393681821275 
LV_cost: 407747.76556488144 
Serv_Trans_cost: 10500 
node_Connection_cost: 40902.34636363637
The total investment cost is estimated at: 484

...

MV_cost: 148361.1 
LV_cost: 320711.46961017745 
Serv_Trans_cost: 10500 
node_Connection_cost: 32969.96927272728
The total investment cost is estimated at: 512542.5388829047 $
The lcoe is: 0.31246492778062873 

The peak load in this cluster is: 131.0745367194644
The area of this cluster is: 2.1608
The max MV load is: 1936.0
Cluster_distance: 29.87955 
HV_km: 0 
MV_km: 31.538228994942582 
LV_km: 169.35306834732074 
Serv_Trans: 3 
nodes: 530.5332490909091
HV_cost: 0 
MV_cost: 167691.14497471292 
LV_cost: 508059.2050419622 
Serv_Trans_cost: 10500 
node_Connection_cost: 53053.32490909091
The total investment cost is estimated at: 739303.6749257661 $
The lcoe is: 0.19060706638036778 

The peak load in this cluster is: 37.11782388658884
The area of this cluster is: 2.16071
The max MV load is: 1936.0
Cluster_distance: 1.1165399999999999 
HV_km: 0 
MV_km: 1.1165399999999999 
LV_km: 70.80229144468129 
Serv_Trans: 3 
nodes: 216.6111781818182
HV_cost: 0 
MV_cost: 15582.699999999999 
LV_cost: 21240

The peak load in this cluster is: 34.68350858184774
The area of this cluster is: 2.02927
The max MV load is: 1936.0
Cluster_distance: 14.137929999999999 
HV_km: 0 
MV_km: 14.137929999999999 
LV_km: 170.41952474965618 
Serv_Trans: 3 
nodes: 545.7082654545455
HV_cost: 0 
MV_cost: 80689.65 
LV_cost: 511258.5742489685 
Serv_Trans_cost: 10500 
node_Connection_cost: 54570.826545454554
The total investment cost is estimated at: 657019.0507944231 $
The lcoe is: 0.46091230995174703 

The peak load in this cluster is: 6.8945830024398544
The area of this cluster is: 2.01939
The max MV load is: 1936.0
Cluster_distance: 5.8153 
HV_km: 0 
MV_km: 5.8153 
LV_km: 20.88132492779502 
Serv_Trans: 3 
nodes: 62.470512727272734
HV_cost: 0 
MV_cost: 39076.5 
LV_cost: 62643.97478338506 
Serv_Trans_cost: 10500 
node_Connection_cost: 6247.0512727272735
The total investment cost is estimated at: 118467.52605611233 $
The lcoe is: 0.4251388501068902 

The peak load in this cluster is: 16.30849894273584
The area of 

Serv_Trans: 3 
nodes: 534.3001454545455
HV_cost: 0 
MV_cost: 101911.5 
LV_cost: 488162.9039494842 
Serv_Trans_cost: 10500 
node_Connection_cost: 53430.01454545455
The total investment cost is estimated at: 654004.4184949388 $
The lcoe is: 0.3322310868959309 

The peak load in this cluster is: 54.83417327624567
The area of this cluster is: 1.9244
The max MV load is: 1936.0
Cluster_distance: 1.0276 
HV_km: 0 
MV_km: 1.0276 
LV_km: 175.0822070906716 
Serv_Trans: 3 
nodes: 575.9979
HV_cost: 0 
MV_cost: 15138.0 
LV_cost: 525246.6212720148 
Serv_Trans_cost: 10500 
node_Connection_cost: 57599.78999999999
The total investment cost is estimated at: 608484.4112720148 $
The lcoe is: 0.30147843862960116 

The peak load in this cluster is: 67.29612049882577
The area of this cluster is: 1.92425
The max MV load is: 1936.0
Cluster_distance: 0.15495 
HV_km: 0 
MV_km: 1.7202075487620678 
LV_km: 134.07513416606574 
Serv_Trans: 3 
nodes: 445.08639090909094
HV_cost: 0 
MV_cost: 18601.03774381034 
LV_cost: 

The total investment cost is estimated at: 469067.9822890787 $
The lcoe is: 0.27571926468051944 

The peak load in this cluster is: 32.14561105290752
The area of this cluster is: 1.8562400000000001
The max MV load is: 1936.0
Cluster_distance: 4.8171800000000005 
HV_km: 0 
MV_km: 4.8171800000000005 
LV_km: 95.79799143274853 
Serv_Trans: 3 
nodes: 318.5958836363636
HV_cost: 0 
MV_cost: 34085.9 
LV_cost: 287393.9742982456 
Serv_Trans_cost: 10500 
node_Connection_cost: 31859.58836363636
The total investment cost is estimated at: 363839.462661882 $
The lcoe is: 0.30598259940069805 

The peak load in this cluster is: 109.69152790773524
The area of this cluster is: 1.85601
The max MV load is: 1936.0
Cluster_distance: 0.8578899999999999 
HV_km: 0 
MV_km: 2.907560034116129 
LV_km: 238.39770209733396 
Serv_Trans: 4 
nodes: 930.4822654545455
HV_cost: 0 
MV_cost: 24537.800170580645 
LV_cost: 715193.1062920019 
Serv_Trans_cost: 14000 
node_Connection_cost: 93048.22654545456
The total investment cos

Cluster_distance: 0.55518 
HV_km: 0 
MV_km: 0.55518 
LV_km: 39.84822673013188 
Serv_Trans: 2 
nodes: 152.47069636363636
HV_cost: 0 
MV_cost: 12775.9 
LV_cost: 119544.68019039564 
Serv_Trans_cost: 7000 
node_Connection_cost: 15247.069636363636
The total investment cost is estimated at: 154567.64982675927 $
The lcoe is: 0.27524447926494927 

The peak load in this cluster is: 17.702891347713052
The area of this cluster is: 0.93076
The max MV load is: 1936.0
Cluster_distance: 26.595090000000003 
HV_km: 0 
MV_km: 26.595090000000003 
LV_km: 51.677002696588346 
Serv_Trans: 2 
nodes: 198.57166000000004
HV_cost: 0 
MV_cost: 142975.45 
LV_cost: 155031.00808976503 
Serv_Trans_cost: 7000 
node_Connection_cost: 19857.166000000005
The total investment cost is estimated at: 324863.6240897651 $
The lcoe is: 0.44887540952205685 

The peak load in this cluster is: 13.77821165761311
The area of this cluster is: 0.93068
The max MV load is: 1936.0
Cluster_distance: 14.75622 
HV_km: 0 
MV_km: 14.75622 
LV_k

## Testing of results

#### First we take a look on general statistics of the results

In [36]:
# This command shows some general statistics on the results
df["lcoe"].describe().transpose()

count    300.000000
mean       0.522362
std        0.848259
min        0.100370
25%        0.223304
50%        0.330863
75%        0.528978
max       11.153439
Name: lcoe, dtype: float64

#### Then we check for extreme values for troubleshooting

In [37]:
x=df["lcoe"].argmax()
print("maximum lcoe achieved achieved in index {x} with the following inputs:\n".format(x=x))
print (df.iloc[x])

people = df.iloc[x]["Pop"]
access_target = df.iloc[x]["ResidentialDemandTierCustom"]
ppl_per_hh = df.iloc[x]["NumPeoplePerHH"]
productive_demand = df.iloc[x]["Productive_demand"]
productive_nodes = df.iloc[x]["Productive_nodes"]
cluster_area = df.iloc[x]["GridCellArea"]
elec_dist = df.iloc[x]["PlannedHVLineDist"]

print ("\n Using these inputs: \n people:{a} \n target: {b} \n ppl_per_hh: {c} \n product_demand:{d} \n product_nodes:{e} \n area:{f} \n distance_from_grid: {g}".format(a=people,b=access_target,c=ppl_per_hh,d=productive_demand,e=productive_nodes,f=cluster_area,g=elec_dist))
print ("\n The results look like this: \n")
test = calc_lcoe(people, access_target, ppl_per_hh, productive_demand, productive_nodes,cluster_area, elec_dist)

maximum lcoe achieved achieved in index 200 with the following inputs:

X_deg                            35.4202
Y_deg                           -15.7972
GridCellArea                     1.99163
AUTO                               14728
Pop                              76.8822
CurrentHVLineDist                21.3657
PlannedHVLineDist                21.3657
TransformerDist                  7.70674
CurrentMVLineDist                7.70674
PlannedMVLineDist                7.70674
ResidentialDemandTierCustom           26
IsUrban                                0
PerCapitaDemand                     43.8
Productive_demand                      0
Productive_nodes                       0
AgriDemand                             0
Conflict                               0
CommercialDemand                       0
NumPeoplePerHH                       5.5
TotalEnergyPerCell               5539.19
MG_Hydro2030                          99
MG_PV2030                       0.753921
MG_Wind2030               

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


## Mapping of electrification results

The following code generates a map showing the calculated cost of electricity at each point. The map can be accessed using the link below.

In [40]:
#Define limits for map rendering
x_ave = df["X_deg"].mean()
y_ave = df["Y_deg"].mean()
lcoe_ave = df["lcoe"].median()

# Create the map using folium module
map_lcoe = folium.Map(location=[y_ave,x_ave], zoom_start=6, control_scale=True)

# Definition of a function that returns different color names based on lcoe result categorization
# Colors are in Hexa-code e.g. #RRGGBB
def colorvalue(x):
    if x <= 0.5:
        return "#ADFF2F"
    elif x >= 0.5 and x < 2:
        return "#32CD32"
    elif x >= 2 and x < 10:
        return "#228B22"
    elif x >= 10 and x < 100:
        return "#008000"
    elif x >= 100 and x < 500:
        return "#006400"
    else:
        return "#000000" 

# The we create a marker for each cluster; 
# We pass coordinates, lcoe value and size as attributes to appear on the rendered map
for index, row in df.iterrows():
    lcoe = row["lcoe"]
    area = row["GridCellArea"]
    color_code = colorvalue(lcoe)
    #radius_size = radius_sizing(area)
    #print (color_code)
    #print (radius_size)
    folium.CircleMarker([row["Y_deg"], row["X_deg"]],
                        radius=2,
                        color=color_code,
                        popup="lcoe: {:.2} USD/kWh, area: {:.2} sq.km".format(row["lcoe"], row["GridCellArea"]),
                        fill = True,
                        fill_opacity=0,
                       ).add_to(map_lcoe)

# We define the limits of the legend and fix its printout format
# We use branca module to create a colormap legend and then add legend to the map
min_lcoe = df["lcoe"].min()
max_lcoe = df["lcoe"].max()
min_lcoe = float("{0:.2f}".format(min_lcoe))
max_lcoe = float("{0:.2f}".format(max_lcoe))
legend = cm.LinearColormap(['#ADFF2F','#32CD32','#228B22','#008000','#006400','#000000'], 
                           index=None, vmin=min_lcoe, vmax=max_lcoe)
legend.add_to(map_lcoe)    

# Create a new directory where the map(s) can be saved
try:
    os.makedirs('maps')
except FileExistsError:
    pass

map_lcoe_output = 'maps/map_{}{}_lcoe.html'.format("Clusters", "Malawi")
map_lcoe.save(map_lcoe_output)

# Finally add the link that leads to the final map output
display(Markdown('<a href="{}" target="_blank">Click here to render the map of electricity cost</a>'.format(map_lcoe_output)))

<a href="maps/map_ClustersMalawi_lcoe.html" target="_blank">Click here to render the map of electricity cost</a>

### The methodology presented below is based on the following sources:

1. Grid Planning for the Future Grid Optimizing Topology and Technology Considering Spatial and Temporal Effects by Hakan Ergun (ABB)
* Generation of Realistic Distribution Grid Topologies Based on Spatial Load Maps by Viktor Lenz ([link](file:///C:/Users/alekor/Box%20Sync/KTH-dESA/Projects/OnSSET/LCoE%20Calculation%20OnSSET/ETH_Thesis_Lenz-SA-2015.pdf))

as well as

3. https://electronics.stackexchange.com/questions/262429/how-many-customers-can-be-connected-to-a-single-wire-earth-return-node-pole-in-a
* https://ieeexplore.ieee.org/document/7343994
* https://library.e.abb.com/public/469882604aaf41bdafd1d391132e6bf9/SSVT_SSMV%20Applications_PPHVITSSVTAPP1115FL.pdf
* https://scholarsmine.mst.edu/cgi/viewcontent.cgi?article=6061&context=masters_theses
* https://pure.tue.nl/ws/files/46798428/449349-1.pdf
* http://www.stonepower.se/Images/SWER.pdf
* https://arxiv.org/pdf/1207.5857.pdf
* https://www.esmap.org/sites/default/files/esmap-files/Rpt_gridextensionesm227.pdf
* https://www-emeraldinsight-com.focus.lib.kth.se/doi/full/10.1108/17506221211259673
* Power Distribution Planning Reference Book Second Edition, Revised and Expanded (Book) by H. Lee Willis

* https://betterexplained.com/articles/techniques-for-adding-the-numbers-1-to-100/

In [ ]:
df